#SQLite com python3
 Nesse código vamos falar sobre os conceitos de conexão com SQLite usando o python3.


 O SQLite é uma biblioteca da Linguagem de programação C no qual fornece um banco de dados bem leve  que não requer um processo de algum servidor separado, e também permite acessar o banco de dados através de uma variante não padrão 
da linguagem de consulta SQL. Ele é uma boa alternativa para se implementar um BD rápido, leve e bem eficiente, principalmente para projetos que precisam fazer testes rápidos ou algo do tipo.


 O módulo sqlite3 que iremos usar para nesse texto 
 foi escrito por Gerhard Häring, no qual fornece uma interface SQL que é compatível com o DB-API 2.0 descrita por PEP 249, criado em
 	29/Mar/2001 por Marc-André Lemburg.


Nesse texto vamos usar um banco de dados colocando no disco para facilitar os nossos exemplos. Você também pode criar um BD na memória RAM, fique a vontade!

Você também pode fornecer o nome especial :memory: para criar um banco de dados na RAM.

In [2]:
import sqlite3
# conexao_BD = sqlite3.connect(':memory:') # na memória RAM
conexao_BD = sqlite3.connect('example_data.db') # fora da memória RAM

cur = conexao_BD.cursor()



Depois de ter um Connection, você pode criar um Cursor objeto e chamar seu método execute() para executar comandos SQL:


In [3]:


cur = conexao_BD.cursor()

# Criar uma tabela

cur.execute(
   '''CREATE TABLE pessoa
             (nome varchar(50), sobrenome varchar(50), email varchar(35), telefone varchar(14));''')

# Inserir dados na tabela criada
cur.execute(''' INSERT INTO pessoa(nome, sobrenome, email, telefone) VALUES 
('Alysson','Araújo','alyssoncontatoaraujo@contato.com','(88)1235-4567'), 
('Jefferson','Andrade','contatojefferson@contato.net','(88)2764-4264'),
('Carlos','Eduardo', 'carloseeduardo@contato.com','(88)8726-4378') ''')

# Salvar (commit) e enviar a modificações e scripts no banco.
conexao_BD.commit()

# Após fazer isso, podemos fechar o banco de dados. Usamos o método close():
# conexao_BD.close()

In [4]:
# outro exemplo de insert:
cur = conexao_BD.cursor()
cur.execute('''INSERT INTO pessoa(nome, sobrenome, email, telefone) VALUES ('Marília','Texeira','mariliatexe@gmaiuil.com','(88)3474-5446')''')
cur.close()


 As formas de fazer consultas no BD é semelhante às outas linguagens, porém o python simplifica bastante o código, facilitando seu entendimento e deixando o código mais limpo.

Para fazer isso precisamos o objeto cursor para usar os métodos SQL, assim como a criação de tabelas, views, etc.., manipulação e comunicação com o bd. 

Colocamos o retorno do método cursor() em uma variável chamada cur.


In [5]:

cur = conexao_BD.cursor()

# Executamos um script sql usando o método execute onde fazemos uma consulta de todos os dados da tabela pessoa 
# ordenada em ordem alfabética pela coluna nome.
for dados in cur.execute('SELECT * FROM Pessoa ORDER BY nome'):
  print(dados)

print('_____________________________________________________________________')
# Outro exemplo seria fazer uma consulta na tabela pessoa apenas nas colunas nome e telefone.
for dados in cur.execute('SELECT nome , telefone FROM Pessoa '):
  print(dados)

print('_____________________________________________________________________')
# Para que os nomes das colunas apareçam, você pode imprimir o nome das colunas antes de imprimir os dados das tabelas. Essa é uma forma, 
# existe outras!
# cur.close()

('Alysson', 'Araújo', 'alyssoncontatoaraujo@contato.com', '(88)1235-4567')
('Carlos', 'Eduardo', 'carloseeduardo@contato.com', '(88)8726-4378')
('Jefferson', 'Andrade', 'contatojefferson@contato.net', '(88)2764-4264')
('Marília', 'Texeira', 'mariliatexe@gmaiuil.com', '(88)3474-5446')
_____________________________________________________________________
('Alysson', '(88)1235-4567')
('Jefferson', '(88)2764-4264')
('Carlos', '(88)8726-4378')
('Marília', '(88)3474-5446')
_____________________________________________________________________


Porém temos que ter cuidado ao usar algumas coisas, como por exemplo em uma situação onde queremos colocar um valor de uma variável no script
SQL em um método do Cursor da seguinte forma: 

In [ ]:

cur = conexao_BD.cursor()
nome = 'Marília'
cur.execute("SELECT * FROM pessoa WHERE nome = '%s'" % nome);

# cur.close()

No exemplo acima, usando operações com string do Python porque fazer isso é inseguro, podendo deixa uma grave vunerabilidade no seu programa para um ataque SQL injection, que em português é injeção de SQ, onde o invasor pode inserir uma instrução personalizda em SQL. Então, tome cuidado!

Para uma melhor segurança, você pode fazer isso usando qmark style, no qual usamos a marcação '?' para dizer que aquele local será passado como parâmetro algo. Segue um exemplo abaixo:



In [ ]:
cur.execute("INSERT INTO Pessoa VALUES (?, ?, ?, ?)", ("João","Almir", "joaoalcontato@contato.com","(88)9273-4214"))

Podemos usar também com named style. Esse é um exemplo de consulta no BD usando esse estilo:

In [ ]:


cur.execute("select * from pessoa where nome=:name", {"name": "Alysson"})
print(cur.fetchall()) # Mais na frente será explicado o fetchall()


### Segue agora, alguns Objetos de cursor do SQLite3:

<br>
<br>

* execute( sql [ , parâmetros ] ) 

  Como já mostrado, o execute executa UM SCRIPT SQL como por exemplo:


In [6]:
for dados in cur.execute('SELECT nome , telefone FROM Pessoa '):
  print(dados)

('Alysson', '(88)1235-4567')
('Jefferson', '(88)2764-4264')
('Carlos', '(88)8726-4378')
('Marília', '(88)3474-5446')


* executescript()

  Porém com o excute, se tentarmos executar mais de um script ele irá dar um warnig. Para evitarmos isso, usamos o executescript() para executar mais de um script de uma vez. Exemplo:

In [ ]:

cur = conexao_BD.cursor()

cur.executescript("""Create table if not exists teste (
  numero integer,
  tipo text,
  situacao text
);

INSERT INTO pessoa(nome, sobrenome, email, telefone) VALUES ('Renam','Texeira','renamttt@gmaiuil.com','(88)5624-5216')
                     ;             
                                  """);
# con.close() caso queira fechar

* fetchall()

  Com o fetchall(), buscamos todas as linhas (restantes) de um resultado de consulta feita anteriormente antes de ser chamado para o uso, onde ele retorna uma lista com os resultados. 

  Uma coisa importante a se falar é que o atributo arraysize do cursor pode afetar o desempenho desta operação, então tenha cautela com o seu uso. 

  Caso nenhuma linha de script SQL tiver sido feita, o fetchall() retorna uma lista vazia.

In [ ]:


cur = conexao_BD.cursor()

print('_____________________________________________________________________')

cur.execute('SELECT nome , telefone FROM Pessoa ')
cur.fetchall()





* fetchone()

  Com ele, fazemos uma busca na próxima linha de um conjunto de resultados de uma consulta feita anteriormente. por exemplo segue a seguinte consulta feita:
~~~~
[('Alysson', '(88)1235-4567'),
 ('Jefferson', '(88)2764-4264'),
 ('Carlos', '(88)8726-4378'),
 ('Marília', '(88)3474-5446'),
 ('João', '(88)9273-4214'),
 ('João', '(88)9273-4214'),
 ('João', '(88)9273-4214'),
 ('João', '(88)9273-4214'),
 ('Renam', '(88)5624-5216')]
~~~~

Quando chamamos fetchone() pela primeira vez, ele irá retornar [('Alysson', '(88)1235-4567')]

Se fizermos isso pela segunda vez, ele retorna [('Jefferson', '(88)2764-4264')] e assim subsequente.


Agora se você comentar o cur.execute('SELECT nome , telefone FROM Pessoa ') depois de executar essa instrução, verá que ele retornará as linhas da consulta, indo de linha em 
linha cada vez que for chamado. Comentar cur.execute('SELECT nome , telefone FROM Pessoa ') faz com que ele seja o último script sql a ser
executado no BD que o curse está trabalhando.


In [12]:
#cur.execute('SELECT nome , telefone FROM Pessoa ')
cur.fetchone()

('Marília', '(88)3474-5446')

Para finalizar, vou mostrar como podemos fazer acesso a colunas usando nomes em vez de índices. Para isso, vamos usar o o sqlite3.row, que é uma classe integrada na qual foi projetada para ser usada como uma forma de construir as  linhas.
Essas linhas que são agrupadas com essa classe podem ser acessadas por índices através de tuplas, sem ter o problema do uso dos nomes das colunas com maiúsculo ou minúsculo nelas, podendo diferenciar o nome original mesmo sendo a mesma. 

Para isso temos o seguinte exemplo do código abaixo que demonstra o uso do Row em uma tabela. Vale lembrar que irei usar o row_factory que, de forma básica, ele consegue alterar este atributo para um chamável  no qual aceita o cursor e a linha original como uma tupla e retornará a linha de resultado real. 

Com isso, podemos implementar maneiras mais avançadas e sofisticadas de retornar resultados, como poder retornar um objeto que também pode acessar colunas por nome.


In [ ]:
import sqlite3

# Dessa vez usarei a memória RAM para alocar o Banco de Dados.
conexao_BD_memory = sqlite3.connect(":memory:")
conexao_BD_memory.row_factory = sqlite3.Row

cur = conexao_BD_memory.cursor()


cur.execute(
   '''CREATE TABLE pessoa
             (nome varchar(50), sobrenome varchar(50), email varchar(35), telefone varchar(14));''')

cur.execute("select 'Alysson' as nome, '(88)1235-4567' as telefone")
for row in cur:
    assert row[0] == row["nome"]
    assert row["nome"] == row["NoME"]
    assert row[1] == row["TeLEFOne"]
    assert row[1] == row["TELEFONE"]

conexao_BD_memory.close()

Referências: 

* [DB-API 2.0 interface for SQLite databases](https://docs.python.org/pt-br/3/library/sqlite3.html)